<a href="https://colab.research.google.com/github/ewapajak/ML/blob/main/Homework06_Tree_methods.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Homework Assignment: Understanding Splitting Criteria in CART for Regression**
---------------------

# **Research Questions**

1. **Equivalence Analysis**  
   Determine whether the above formulations are equivalent or if they can yield different split choices. Specifically:
   - Are *local RSS minimization* and *RSS gain maximization* equivalent?
   - Does *total RSS minimization* coincide with either of these two, or is it distinct?

##**Local RSS minimization and RSS maximization**:

**Local RSS minimization**:
We aim to choose the feature $X_j$ and threshold $t$ that minimizes the following:

 $$ (X_j^*, t^*) = \arg\min_{X_j, t} \sum_{m=1}^{2} \sum_{i : x_i \in R_m(X_j, t)} (y_i - \bar{y}_{R_m})^2.
   $$

To put it simply, we evaluate all possible splits of our data and then choose such that the Residual Sum of Squares is minimized (the predictions are as close as possible to the mean response of their respective subsets).

**RSS Gain Maximization**:

We aim to choose the feauture $X_j$ and threshold $t$ that maximizes the following:
 $$
   (X_j^*, t^*) = \arg\max_{X_j, t} \Bigl\{
   \underbrace{\sum_{i : x_i \in \text{Parent}} (y_i - \bar{y})^2}_{\text{Parent RSS}}
   \;-\;
   \underbrace{\sum_{m=1}^{2} \sum_{i : x_i \in R_m(X_j, t)} (y_i - \bar{y}_{R_m})^2}_{\text{Children RSS}}
   \Bigr\}.
   $$

The **RSS gain** is the difference between the RSS of the parent node and the RSS of the children nodes.


The maximization of RSS gain and minimization of local RSS are indeed equivalent; for any given node the parent RSS is constant, therefore when maximizing the gain, we want to minimize the term that we are substracting, which is the sum of the RSS of the children nodes:

- $$
\text{RSS}_{\text{parent}} = \sum_{i \in R} (y_i - \bar{y}_R)^2
$$

- $$
\text{RSS}_{\text{children}} = \sum_{i \in R_1} (y_i - \bar{y}_{R_1})^2 + \sum_{i \in R_2} (y_i - \bar{y}_{R_2})^2
$$

- $$
\text{Gain} = \text{RSS}_{\text{parent}} - \text{RSS}_{\text{children}}
$$




where $R_1$ and $R_2$ are the two children regions after the split, $\bar{y}_R$ is the mean of all $y_i$, and $\bar{y}_{R_1}, \bar{y}_{R_2}$ - the mean of $y_i$ in regions $R_1$, $R_2$ respectively.










##**Total RSS minimization**:
The objective of **minimizing total Residual Sum of Squares (total RSS)** is stated as:
   $$
   (X_j^*, t^*) = \arg\min_{(X_j, t)} \sum_{m \in \mathrm{Leaves}(T(X_j, t))}
   \sum_{i \in R_m} \Bigl(y_i - \overline{y}_{R_m}\Bigr)^2,
   $$
   where
   $$
   \overline{y}_{R_m} = \frac{1}{\lvert R_m \rvert}
   \sum_{i \in R_m} y_i
   $$
   is the mean response in leaf $m$.

The local RSS is the sum of RSS in the two child nodes after the split, while the total RSS is the usm of RSS values across all leaves in the final tree. During local RSS minimization we are choosing the best split for one node, without considering the rest of the tree. During total RSS minimization we are looking at the entire tree trying to split our data so that the final total error over *all*  leaves is minimized. Without any pre-pruning, minimizing the RSS locally and globally would lead to the same results, as the greedy algorithm minimizing the RSS locally explores all optimal splits. However, when constraints limiting tree growth are introduced - such as`max_depth` or `min_samples_split`, the results may differ. For example, under a minimum depth constraint, the greedy algorith may find splits that are locally optimal, but it might not be able to continue splitting in a way that minimizes total RSS globally. As a result, the tree can get 'stuck' on a suboptimal structure, with higher total error than a tree without such limitations.



2. **Empirical Experiment**  
   Design and conduct a Python experiment to determine which of these formulations is implemented in `scikit-learn` in `DecisionTreeRegressor`. Present numerical results and plots to support your conclusion.

   

In [6]:
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import export_text


#dataset
X = np.array([[1], [2], [3], [4], [5]])
y = np.array([1,2,4,7,9])

reg = DecisionTreeRegressor(max_leaf_nodes = 3, random_state = 0)
reg.fit(X,y)

print(export_text(reg, feature_names = ["X"]))

|--- X <= 3.50
|   |--- X <= 2.50
|   |   |--- value: [1.50]
|   |--- X >  2.50
|   |   |--- value: [4.00]
|--- X >  3.50
|   |--- value: [8.00]



**The results**:
The tree splits at values $2.5$ and $3.5$, the leaves have values $1.5, 4.0$ and $8.0$. The results suggest that scikit-learn is using local RSS minimization, since it seems to aim to reduce RSS in each step, without trying to optimize the global result.